In [ ]:
import os
from utils.denoising_utils import *

FOLDER = "BSDS300/images/train/"
IMAGES = [FOLDER + img for img in os.listdir(FOLDER)][:]
def L2(A):
    A_1D = np.reshape(A, (-1))
    return np.sqrt(A_1D.dot(A_1D) / A_1D.shape[0])
 

In [ ]:
coeffs_se = [-3.1389654e-6, -7.0210049e-1, -1.8598057, 3.9881199, -8.3760888, 9.7330082, -6.9596693, 2.9464712, -7.3358565e-1, 9.9630886e-2, -5.7155088e-3]
coeffs_sm = [1.6923658e-1, 7.0309281e-1, 3.7234715e-2, 2.4377832e-2, 2.0282884e-3, -1.7851033e-5, -8.5123452e-5, -2.6295693e-5, 3.2172868e-7, 1.4308530e-6]

import torch
dtype = torch.cuda.FloatTensor
# p = np.polynomial.Polynomial(coeffs)
def p_se(x):
    return sum([(x**p)*coeff for p, coeff in enumerate(coeffs_se)])

def p_sm(x):
    return sum([(x**p)*coeff for p, coeff in enumerate(coeffs_sm)])

def Se(ksi):
    if not torch.is_tensor(ksi):
        ksi = dtype([ksi])
    return 1 - torch.exp(p_se(torch.sqrt(ksi)))

def Sm(mu):
    if not torch.is_tensor(mu):
        mu = dtype([mu])
    return (1+torch.tanh(p_sm(mu)))/2

def clipped_var(image, variance):
    return variance*Se(image/variance)*Se((1-image)/variance)# + 0.5*variance*Se(image/variance)*Se((1-image)/variance)

def rms(values):
    return torch.sqrt((torch.sum(values*values))/values.nelement())

def average_clipped_var(image, variance):
    image = torch.from_numpy(image).type(dtype)
    pred = torch.sqrt(torch.sum(clipped_var(image, variance)**2)/image.nelement())# + 0.6*variance**2 - 0.005 * variance
    # coeffs = [0.018521612453785252, 0.50432376, 1.40543164, 0.16467695, 0.21288617] # no cross
    # coeffs = [0.00470733087124342, 0.9962607806987671, -2.5801294201977707, -0.06255021879385336, -0.6016675682100312, 3.6189465922690225]
    # coeffs = [-0.021957900592563084, 1.918386322644575, -7.04142476640072]
    coeffs = [0.00010340009150736074, 1.0680248634648952, -2.9185771099300277, -0.06193161972243111, -0.6269524747313732, 3.7192874183112843]
    vars_ = [1, pred, pred**2, variance, variance**2, pred*variance]
#     vars_ = [1, pred, pred**2]
    return sum([c * v for c, v in zip(coeffs, vars_)])
#     return corrected
    return pred
    #     return torch.mean(clipped_var(image, variance))

In [ ]:
clipped_var(1.0, 100/255)*255

In [ ]:
n = 1000000
sigma = 50
v = np.full(n, 1.0)
noise = np.random.normal(loc=np.full(n, 0), scale=sigma/255)
clipped = np.clip(v + noise, 0.0, 1.0)
# print(255*np.abs(-v + clipped))
print(255*L2(-v + clipped))
print(255*average_clipped_var(clipped, sigma/255))
# print(clipped)
# print(noise)

In [ ]:
stddev = []
mean = []
sigmas = np.arange(5,105,5)
sigmas = [25,50,75,100]
sigmas = [25]
bigdata = []
handles = []
for sigma in sigmas:
    print(sigma)
    sigma = float(sigma)
    data = []
    for fname in IMAGES:
        img_pil = crop_image(get_image(fname, -1)[0], d=32)
        img_np = pil_to_np(img_pil)
        _, img_noisy_np = get_noisy_image(img_np, sigma/255, clip=True)
        diff_n = img_noisy_np - img_np


        computed = diff_n.std()*255
        expected = 255*average_clipped_var(img_noisy_np, sigma/255)
#         data.append((computed, float(expected)))
        data.append((computed, float(expected), sigma))
#         print("Computed:\t{:.2f}\tPredicted:\t{:.2f}\tDelta:  {:.2f}\tvar:\t{:.2f}\tmean:\t{:.2f}".format(
#             computed, expected, computed-expected, img_noisy_np.std(), img_noisy_np.mean()
#         ))
    handles.append(plt.scatter([x[0] for x in data], [float(y[1]) for y in data], label="σ = {}".format(sigma)))
    m, M = min([x[0] for x in data]), max([x[0] for x in data])
    plt.plot([m, M], [m, M], color="k", linestyle='-', linewidth=2)
    bigdata.extend(data)
#     plt.show()
plt.xlabel("Measured standard deviation of the method noise")
plt.ylabel("Predicted standard deviation of the method noise")
plt.legend()
plt.savefig(fname="mn_pred.pdf", format='pdf')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
bigdata_ = bigdata
# poly PolynomialFeatures(degree=2)
lin = LinearRegression(fit_intercept=True)
bigdata_norm = [[x/255 for x in row] for row in bigdata_]

X = [[x[1], x[1]**2, x[2], x[2]**2, x[1]*x[2]] for x in bigdata_norm]
# X = [[x[1], x[1]**2] for x in bigdata_norm]
Y = [x[0] for x in bigdata_norm]

lin.fit(X, Y)
print([lin.intercept_] + list(lin.coef_))
print(lin.score(X, Y))


In [ ]:
bigdata_